<a href="https://colab.research.google.com/github/Rociorj18/Abitofasta/blob/main/Script_abitoconsenso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AB1 -> FASTQ -> ALINEAMIENTO

Este script tiene como objeto coger los archivos ab1 que nos mandan de la secuenciación y pasarlo a fastq (nos permite ver la calidad de la posicion de las diferentes bases). Tras esto, la secuencia reverse la convierte en la reverse complement y genera un alineamiento por pares entre secuencias que se llamen igual, es decir, al forward y la reverse complement.

- **INPUT** :
  
  (1) `LOS ARCHIVOS AB1` : DEBES METER UN NUMERO PAR DE SECUENCIAS PORQUE VAN PAREADAS REVERSE Y FORWARD
- **OUTPUT** :
**(1) `ALINEAMIENTO` : DONDE ESTA LA FORWARD Y LA REVERSE*
**(2) `CONSENSO`: SOLA GENERADA A PARTIR DE FORWARD Y REVERSE*
**(3) `CONSENSUS` CON TODAS LAS CONSENSO DE LA CARPETA*

In [ ]:
# instalar libreria biopython
!pip install biopython

In [ ]:
# instalar diferentes paquetes
from Bio import SeqIO, pairwise2
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import Align
import os
import shutil

def read_abi(abi_file: str, reverse: bool): # creamos una nueva función
# llamada read_abi, toma la ruta del archivo abi y reverse para indicar si debemos o no invertir la secuencia
    records = SeqIO.parse(abi_file, "abi")
    record = list(records)[0]
    if reverse:
        rev_record = record.reverse_complement()
        rev_record.letter_annotations = {
            'phred_quality': list(reversed(record.letter_annotations["phred_quality"]))
        }
        rev_record.id = record.id
        rev_record.description = 'reverse_complement'
        record = rev_record

    return record

def make_consensus(aln, consensus_name): # otra funcion para crear el alineamiento
    target_score_idx = 0
    query_score_idx = 0
    consensus_seq = '' # crea una cadena vacia para las consenso
    print(aln.format("fasta"))
    print(aln.target.letter_annotations["phred_quality"])
    print(aln.query.letter_annotations["phred_quality"])
    # aln[0] is the target(la que se usa como referencia) and aln[1] is the query
    for j in range(aln.length):
        if aln[0][j] == aln[1][j]: # si las bases son iguales en las dos cadenas
            consensus_seq += aln[0][j] # agrega esa base
            query_score_idx += 1 # suma 1 al query score
            target_score_idx += 1 # suma 1 al targ
        elif aln[0][j] == '-': # si la target es tiene gap,
            consensus_seq += aln[1][j] #   pon la de la query
            query_score_idx +=1 # suma un punto al query
        elif aln[1][j] == '-': # si es la query la que tiene gap
            consensus_seq += aln[0][j] # pon la del target
            target_score_idx += 1 # sumale un punto
        else:
            # si ambas son diferentes, comparo el phred_quality de la target, y de la query, y si el que sea
            # mayor lo pongo, ########## ALERTA, SI ES MAYOR IGUAL LA target se pone
            if aln.target.letter_annotations["phred_quality"][target_score_idx] >= \
                    aln.query.letter_annotations["phred_quality"][query_score_idx]:
                consensus_seq += aln[0][j]
            else:
                consensus_seq += aln[1][j]

            query_score_idx += 1 # sumammos
            target_score_idx += 1 # sumamos

    return SeqRecord(Seq(consensus_seq), id= consensus_name, description="")

# si quieres cambiar el archivo de salida
output_folder = "/content/resultados_fasta" # los output van a esta carpeta
os.makedirs(output_folder, exist_ok=True) # si no existe, la creo

# crea lista vacia llamada forwards y reverse para almacenar las rutas de los archivos de secuencias
forwards = []
reverses = []

# ================================================= IMPORTANTE: DONDE CAMBIAR NOMBRE DE DIRECTORIO Y CONDICION PARA ARCHIVOS ==============================================================


seqs_dir = "/content/EJEMPLO" ### cambiar nombre si usas otro directorio
### CAMBIAR CONDICION SI NOMBRE DE ARCHIVO ES DIFERENTE
for f in os.listdir(seqs_dir): ## intera sobre los archivos en el directorio especificado (seqs_dir)
    if f.count('_') == 2: # si el nombre del archivo contiene dos "_"
        reverses.append(f'{seqs_dir}/{f}') # se considera reverse y se añade a la lista de reverse
    else:
        forwards.append(f'{seqs_dir}/{f}') # si no, forwards

# se ordena alfabeticamente las listas
forwards = sorted(forwards)
reverses = sorted(reverses)

if len(forwards) != len(reverses): # verifica si la cantidad de secuencias forward es igual que la de reverse, si no es
# lanza un aviso, y para el script
    raise Exception('The number of forward and reverse chromatograms must match!')

aligner = Align.PairwiseAligner(scoring='blastn') # el scoring indica que se utiliza un esquema de puntuación similar al
# utilizado en BLAST para alinear.

consensuses = [] # crea una lista

for i in range(len(forwards)): # itera sobre toda la longuitud de secuencias
    forward_seq = read_abi(forwards[i], False) # lee el archivo abi y el parametro reverse lo pone en false para leer la secuencia normal.
    reverse_seq = read_abi(reverses[i], True) # en este caso leera la secuencia en su orientacion inversa
    aln = aligner.align(forward_seq, reverse_seq)[0] # alinea las seucncias, devuelve una lista de los alineamientos y se obtiene el primer alineamiento con el [0]

    # Extraer el nombre común de las muestras
    common_name = os.path.basename(forwards[i]).split('_')[0]
    consensus_seq = make_consensus(aln, common_name)
    consensuses.append(consensus_seq)

    # Guardar archivo de consenso en formato FASTA con límite de 60 caracteres por línea
    consensus_filename = os.path.join(output_folder,f'{common_name}_consensus.fasta')
    with open(consensus_filename, 'w') as consensus_file:
        consensus_file.write(f">{consensus_seq.id}\n")
        sequence = str(consensus_seq.seq)
        for j in range(0, len(sequence), 60):
            consensus_file.write(sequence[j:j+60] + '\n')

    # Guardar archivo de alineamiento en formato FASTA
    aln_filename = os.path.join(output_folder, f'{common_name}_aln.fasta') # arhcivo alineamiento
    with open(aln_filename, 'w') as f:
        f.write(f"{aln.format('fasta')}")
        f.write(f">{consensus_seq.id}\n")
        f.write(str(consensus_seq.seq))

# Output FASTA file name
consensuses_fasta = "consensuses.fasta" # Si quieres cambiar nombre del archivo final del consenso

# Write each SeqRecord to the FASTA file
with open(consensuses_fasta, "w") as output_handle: # write (genera un fichero nuevo aunq ya exista), r read, a append, coge un fichero y añade modificandolo
    SeqIO.write(consensuses, output_handle, "fasta") #nuestra lista

# Comprimir la carpeta resultados_fasta como un archivo zip
shutil.make_archive(output_folder, 'zip', output_folder)


/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


>21077_Premix
--------ACAAAGTTTCAGATGACTGGGTAACGTACAGTACTTTTGTGTTTACGA---GAAC-CAAGATTT-CCAGAT-GGACGGGGTA--GAGAAAGGTTCTTGTGTATTTGCGCAAACCATTATTTAATATTAGAATCGGAAAAATCAGCCCG-GCCGGCTTACTAATGGGATGCCCTAATACATTTGAAAATTTCGCTTTAGCCAATGACGCTATCCTTGGACTAACTGGAACAAGGGTGTCGAACTTTCTTATAGAATTATTGTTTAGAAATTCTTGTGCTAGAATTTGAGTCCGTAACCCCAAAAGGACTCGTCGGAACGCTTGAAAGATAGCCCAAAAAATCAAGGAAATTGTGGGCTAATTGGTTTATATAAATCCTTTTTGGATGAAACCACAGAGAAAAGTGCCATTGCCAAAAAGTGACAAGGTAACATTTCCATTTATTCATCAAAAGAGACGTCCCTTTTGAAGCCAGAATGGATGTTCTTTGATACCTAACATAATGCATGCAAGGTTCTTTGACCAACCATGGATTCGCCTGAAAATCCTTAACCTTAACAAAGACGTTCACAAGACGTTCTATTTTTATATAGAAATAGATTCGTTCAAGAAGAACTCCAGAAGATGTTGATCGTAAAT--GAGAAGATTGGTTACATAGAAAGACGAAAATGGATTCATATTCACATACATGAGAATTATATAAGAATAAGAATAATCTTTGATTTTTTTTTGAAAAATGGAAACTGGCTTTCTTTGAAATAATAAGACTATTCCAATTACAATACTCGTTCAGAAAGAATCGTAATAAATGCAAAGAACAGGCTTCTTTTACCCAATAGGAAAAAGATTTATACAAGAATTCCAGATGGACTGGGTAAAAGAAATACCACCATCCTACGTTTTATGATTTGACGTTTACTTGATGTACTAGATACATCCCATTATTCTGATATTGACCCTTTGTATTTATTGCTGAAG

'/content/resultados_fasta.zip'

# Para interpretar el print:
Los numeros que salen son el  **phred score**
Como este score no cuenta los gaps y como ves, la primera secuencia tiene 8 gaps, para mirar el phred score del primer nucleotido (a) tendrias que mirar el primer numero (6), pero para la segunda secuencia tendrias que ir a la posición correspondiente de esa A que es la posicion  9 con un phred score de (5).

Para la A de la posicion 3 phred (8), tendrias que ir con la posicion 10 del otro que seria (5).
El que tenga el phred score más alto es el que gana y por tanto impone su nucleotido.
!!!!!!!!!! CUIDADO PORQUE EN LA SECUENCIA DE ABAJO TAMBIEN SALTA  EL PHRED SCORE SI HAY UN GAP!!!!!!